# Normalized Difference Water Index (NDWI) 

Articulo de referencia
 
https://www.sciencedirect.com/science/article/abs/pii/S0034425796000673

El Índice de Agua de Diferencia Normalizada (NDWI) fue desarrollado por Gao (1996) como un índice del contenido de agua de la vegetación. El índice es sensible a los cambios en el contenido líquido de los doseles vegetales. Esto significa que el índice se puede utilizar, por ejemplo, para detectar vegetación que experimenta condiciones de sequía o diferenciar los niveles de riego de los cultivos. En áreas secas, los cultivos que se irrigan se pueden diferenciar de la vegetación natural. También se denomina a veces Índice de Humedad de Diferencia Normalizada (NDMI).



**How it Works:**
 
**Healthy Vegetation:**
- High NIR reflectance (due to leaf structure).
- Low SWIR reflectance (water strongly absorbs SWIR).
- Result: Positive NDWI values indicate high water content.
 
**Stressed or Dry Vegetation:**

- NIR reflectance decreases slightly, but SWIR reflectance increases significantly (less water absorption).
- Result: NDWI values decrease, approaching zero or negative, indicating low water content.


**Interpretation:**
- High NDWI (Near 1) – Vegetation with high water content (healthy).
- Moderate NDWI (0 to 0.5) – Vegetation with moderate water content.
- Low NDWI (Negative values) – Stressed or dry vegetation


**Applications:**
- Agriculture – Monitor crop water stress and optimize irrigation.
- Forestry – Assess drought conditions in forests and detect forest fires.
- Climate Studies – Track large-scale vegetation response to climate change.
- Disaster Management – Identify areas of vegetation at risk of drought or fire.

La formula del NDWI es la siguiente: 

<div style="text-align: center;">
    <img src="Images/NDWI formula.png" alt="Descripción de la imagen" width="500">
</div>
 

**BANDAS NECESARIAS:** 
 
En este caso utilizaremos imagenes de Sentinel 2. Por ende, hay que encontrar cuales son las bandas en sentinel 2 que corresponden a estas longitudes de onda. 

En el caso de Sentinel 2 la banda NIR (Near Infrared) es la banda **8** con una longitud de onda de Central Wavelength = 842nm.  Por otra parte, la banda de SWIR (short-wave infrared) es mejor el uso de la banda 11 que tiene un centro de longitud de onda  = 1610nm. 

**NOTA:** El rnago de SWIR adecuados aparentmenete son entre 1,400 a 1,900 nm. 



**IMPORTACION DE LIBRERIAS**

In [1]:
import os 
import ee 
import geemap
import pandas as pd 
import requests

**AUTENTIFICACION**

In [2]:
# 1) Obtención de la dirección de trabajo 
direction = os.getcwd()

# Remove the last part of the path
direction = os.path.dirname(direction)

# Dirección de la llave 
service_account = direction + '/conf/local/gcp-for-data-science-397913-4fd843feede1.json'

# Autentificación 
credentials = ee.ServiceAccountCredentials(email=None, key_file=service_account)
ee.Initialize(credentials)

## 1) Lectura y procesamiento de imagenes  
Las imagenes satelitales se almacenaron en un bucket de GCP, por lo que serán leídas de ahi mismo.

API Reference:
 
  https://developers.google.com/earth-engine/apidocs/ee-image-loadgeotiff

### 1.1) Cálculo y exportación de imagenes con NDWI

In [3]:
# 1) Lectura del nombre de las imagenes en el bucket 
images_names = pd.read_csv(filepath_or_buffer=direction + "/data/02_intermediate/images_names.csv")

# Diccionario vacio, aqui almacenaremos los resultados 
NDWI ={}

# Creación del mapa base 
Map = geemap.Map()

# Path to the image in the GCS bucket
image_center = images_names["Images names"][0]
image_center   = f'gs://earth_engine_selene/sentinel2/{image_center}.tif'
    
# Load the image from the GCS bucket
image = ee.Image.loadGeoTIFF(uri=image_center)

# Central el mapa 
Map.centerObject(ee_object=image, zoom= 9)

# 2) Inicio del ciclo 
for i in images_names["Images names"]:
    
    # 1) Descarga de imagenes 
    # Path to the image in the GCS bucket
    gcs_path = f'gs://earth_engine_selene/sentinel2/{i}.tif'
    
    # Load the image from the GCS bucket
    image = ee.Image.loadGeoTIFF(gcs_path)
      
    # 2) Calculate NDVI (if bands are organized)
    image = image.normalizedDifference(bandNames=['B8', 'B11']).rename('NDWI')  # Renombre de la banda resultante (capa) 
    print(f"NDWI Calculated for {i}")
    
    # Define visualization parameters
    vis_params = {
        'min': -0.5,
        'max': 1,
        'palette': ['brown', 'yellow', 'green']
    }
    
    url = image.getThumbURL(params=vis_params)
    print(f"Thumbnail URL for {i}: {url}")
        
    # 4) Descarga de los datos 
    # Download the image and save it locally
    output_file = direction + f"/data/07_model_output/NDWI_{i}.png"
    
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_file, 'wb') as f:
            f.write(response.content)
        print(f"Image successfully downloaded as '{output_file}'")
    else:
        print(f"Error: Unable to download the image. HTTP status code {response.status_code}")
        
    # 5) Almacenamiento de resultados en un diccionario 
    NDWI[i] = image
    
    # 6) Añadimos la imagen al mapa html 
    Map.addLayer(ee_object=image,  vis_params= vis_params, name=f'NDWI {i}', shown=False)
    
# Exportación del HTML 
html_file = direction + "/data/07_model_output/Map_NDWI.html"
Map.to_html(filename=html_file, title="My Map", width="100%", height="880px") # Exporacion a una html 

NDWI Calculated for image_2023-1
Thumbnail URL for image_2023-1: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/3b23aba317e5633080cf0ac949b88039-6a649921b277d67aee8db69fcf637ae7:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/NDWI_image_2023-1.png'
NDWI Calculated for image_2023-3
Thumbnail URL for image_2023-3: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/20fed02535b5f4b46243cb4ad9d0a29f-fea028f18ff75b9c236ea6f216d176a0:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/07_model_output/NDWI_image_2023-3.png'
NDWI Calculated for image_2023-5
Thumbnail URL for image_2023-5: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/8403922e1df202040c56e58c0bf15b34-70195376b41e885c768272fb34ac828c:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios